# Visualizing Provenance from a Semantic Container

In this demo we will look at the provenance information from a semantic container. The provenance information can be access along with the data and usage policy in `/api/data/provision`. 

In [1]:
prov_url = "https://vownyourdata.zamg.ac.at:9505/api/data/provision"

In [2]:
import requests

In [3]:
r = requests.get(prov_url)

The request will return a JSON document, where the provenance information is available in the `$.provenance` path of the document.

In [4]:
provision = r.json()['provenance']

The provenance is described using RDF format using PROV-O Ontology. Therefore we will need to use `rdflib` library to access it.

In [5]:
from rdflib import ConjunctiveGraph as Graph

In [6]:
g = Graph()
g.parse(data=provision, format="trig")
len(g)

2115

Now let's have a look at the provenance information. We can just use TURTLE serialization format for human-readable RDF document.

In [7]:
print(g.serialize(format="turtle").decode("utf-8"))

@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sc: <http://w3id.org/semcon/ns/ontology#> .
@prefix scr: <http://w3id.org/semcon/resource/> .
@prefix void: <http://rdfs.org/ns/void#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

scr:input_0077304d8b68 a prov:Activity ;
    rdfs:label "input data from 2019-04-19T00:00:12Z"^^xsd:string ;
    sc:inputHash "0077304d8b686e2db6bb96cdcbe7b5c5245966607f3054b4b7daa5cbe7700ee8"^^xsd:string ;
    prov:endedAtTime "2019-04-19T00:00:12+00:00"^^xsd:dateTime ;
    prov:generated scr:data_745aa45c1bf1_e6dd0a07 ;
    prov:startedAtTime "2019-04-19T00:00:12+00:00"^^xsd:dateTime ;
    prov:used scr:data_bfa14dbae93c_e1219b99 .

scr:input_02d722baad4b a prov:Activity ;
    rdfs:label

The provenance document contains information on the data addition event as `prov:Activity`, the involved parties are modeled using `prov:Agent` or the subclasses (`prov:SoftawareAgent`, `prov:Organization`), and each data item that associated with each data modification activity as `prov:Entity`. 

For easier viewing, we can use [PROV-O Viz](http://provoviz.org) to generate visualization of this provenance information. We will use HTTP POST request containing the provenance information. 

In [8]:
rv = requests.post("http://provoviz.org/service", data={'graph_uri':prov_url, 'data':g.serialize(format="turtle")})
rv

<Response [200]>

The response of this request contains HTML string, but in order to show it here we will use `IFrame` with data URL (otherwise the javascript won't work). 

In [9]:
from IPython.display import IFrame
import base64
IFrame(src="data:text/html;base64,"+base64.b64encode(rv.text.encode("utf-8")).decode("ascii"), width="100%", height=400)